<a href="https://colab.research.google.com/github/VKSi/2021_03_GB_NLP/blob/main/HW5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации  
2. написать свой теггер как на занятии, но улучшить попробовать разные векторайзеры, добавить знание не только букв и слов но и совместно объединить эти признаки  
3. вместо векторайзеров взять эмбединги попробовать (word2vec и fasttext по желанию дополнительно можно взять tf.keras.layers.Embedding)  
4. взять не только эмбединги каждого слова, но и взять соседей, т.е. информацию о соседях количество соседей выбрать самим (узнать наилучшее количество соседей)    
5. сравнить все реализованные методы сделать выводы

### загрузка данных

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install pyconll

In [ ]:
import pyconll

In [ ]:
!mkdir datasets

mkdir: cannot create directory ‘datasets’: File exists


In [ ]:
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2021-04-18 09:49:11--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81043533 (77M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train.conllu’

./datasets/ru_synta 100%[===================>]  77.29M  95.2MB/s    in 0.8s    

2021-04-18 09:49:15 (95.2 MB/s) - ‘./datasets/ru_syntagrus-ud-train.conllu’ saved [81043533/81043533]

--2021-04-18 09:49:15--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercon

In [ ]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [ ]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [ ]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

### 1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации  

In [ ]:
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger

In [ ]:
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(fdata_sent_test[100]), unigram_tagger.evaluate(fdata_test))

[('Это', 'PRON'),
 ('сочинение', 'NOUN'),
 ('известно', 'ADJ'),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'ADV'),
 ('более', 'ADV'),
 ('древним', 'ADJ'),
 ('рукописям', None),
 ('XVI', 'NUM'),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.8772368820139521

In [ ]:
bigram_tagger = BigramTagger(fdata_train)
display(bigram_tagger.tag(fdata_sent_test[100]), bigram_tagger.evaluate(fdata_test))

[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', None),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', None),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', None),
 ('к', 'ADP'),
 ('ещё', None),
 ('более', 'ADV'),
 ('древним', None),
 ('рукописям', None),
 ('XVI', None),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.6963232568328109

In [ ]:
trigram_tagger = TrigramTagger(fdata_train)
display(trigram_tagger.tag(fdata_sent_test[100]), trigram_tagger.evaluate(fdata_test))

[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', None),
 ('во', 'ADP'),
 ('многих', None),
 ('вариантах', None),
 ('(', None),
 ('самые', None),
 ('ранние', None),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', None),
 ('лет', None),
 ('старше', None),
 (')', None),
 ('и', 'PART'),
 ('восходит', None),
 ('к', None),
 ('ещё', None),
 ('более', None),
 ('древним', None),
 ('рукописям', None),
 ('XVI', None),
 ('в', 'ADP'),
 ('.', None)]

0.24808748694099012

In [ ]:
bigram_bo_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_bo_tagger.tag(fdata_sent_test[100]), bigram_bo_tagger.evaluate(fdata_test))

[('Это', 'PRON'),
 ('сочинение', 'NOUN'),
 ('известно', 'ADJ'),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'ADV'),
 ('более', 'ADV'),
 ('древним', 'ADJ'),
 ('рукописям', None),
 ('XVI', 'NUM'),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.8829996966939642

In [ ]:
trigram_bo_tagger = BigramTagger(fdata_train, backoff=bigram_bo_tagger)
display(trigram_bo_tagger.tag(fdata_sent_test[100]), trigram_bo_tagger.evaluate(fdata_test))

[('Это', 'PRON'),
 ('сочинение', 'NOUN'),
 ('известно', 'ADJ'),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'ADV'),
 ('более', 'ADV'),
 ('древним', 'ADJ'),
 ('рукописям', None),
 ('XVI', 'NUM'),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.8829996966939642

** Вывод **     
 Увеличение "граммности" по отдельности резко ухудшает качество.     
 Применени трехграмного таггера с бэкофом к предыдущим тагерам НЕ повышает кчества.     
 Лучшее качество достигнуто на биграммном тагере с обратным вызовом юниграммного

### 2. написать свой теггер как на занятии, но улучшить попробовать разные векторайзеры, добавить знание не только букв и слов но и совместно объединить эти признаки

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [ ]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [ ]:
test_enc_labels = le.transform(test_label)

In [ ]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [ ]:
def task_1_2(vectorizer, _train_tok=train_tok, _test_tok=test_tok, _train_enc_labels=train_enc_labels):
  X_train = vectorizer.fit_transform(train_tok)
  X_test = vectorizer.transform(test_tok)
  lr = LogisticRegression(random_state=0)
  lr.fit(X_train, train_enc_labels)
  pred = lr.predict(X_test)
  return accuracy_score(test_enc_labels, pred)

In [ ]:
print(task_1_2(HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=100)))

0.685454790550332


In [ ]:
print(task_1_2(HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=100)))

0.288326087689145


In [ ]:
print(task_1_2(HashingVectorizer(ngram_range=(1, 5), analyzer='char_wb', n_features=100)))

0.7333265931992047


In [ ]:
print(task_1_2(HashingVectorizer(ngram_range=(1, 2), analyzer='word', n_features=100)))

0.2888989990900819


In [ ]:
print(task_1_2(HashingVectorizer(ngram_range=(1, 2), analyzer='char_wb', n_features=100)))

0.7702119772183467


In [ ]:
print(task_1_2(CountVectorizer(ngram_range=(1, 5), analyzer='char', max_features=100)))

0.7769689616823375


In [ ]:
print(task_1_2(CountVectorizer(ngram_range=(1, 2), analyzer='char_wb', max_features=100)))

0.7803306035790112


In [ ]:
print(task_1_2(CountVectorizer(ngram_range=(1, 1), analyzer='word', max_features=100)))

0.382081353418933


In [ ]:
print(task_1_2(TfidfVectorizer(ngram_range=(1, 5), analyzer='char', max_features=100)))

0.7747110167492333


In [ ]:
print(task_1_2(TfidfVectorizer(ngram_range=(1, 2), analyzer='char_wb', max_features=100)))

0.7830098068951572


In [ ]:
print(task_1_2(TfidfVectorizer(ngram_range=(1, 1), analyzer='word', max_features=100)))

0.382081353418933


In [ ]:
import scipy.sparse as sp

In [ ]:
vectorizers = [HashingVectorizer(ngram_range=(1, 2), analyzer='char_wb', n_features=100),
               HashingVectorizer(ngram_range=(1, 2), analyzer='word', n_features=100),
               CountVectorizer(ngram_range=(1, 2), analyzer='char_wb', max_features=100),
               CountVectorizer(ngram_range=(1, 2), analyzer='word', max_features=100),
               TfidfVectorizer(ngram_range=(1, 2), analyzer='char_wb', max_features=100),
               TfidfVectorizer(ngram_range=(1, 1), analyzer='word', max_features=100)
               ]

for i, vectorizer in enumerate(vectorizers):
  X_tn = vectorizer.fit_transform(train_tok)
  X_tt = vectorizer.transform(test_tok)
  if i == 0:
    X_train, X_test = X_tn, X_tt
  else:
    X_train = sp.hstack((X_train, X_tn), format='csr')
    X_test = sp.hstack((X_test, X_tt), format='csr')

lr = LogisticRegression(random_state=0)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.855348296431099

### 3. вместо векторайзеров взять эмбединги попробовать (word2vec и fasttext по желанию дополнительно можно взять tf.keras.layers.Embedding)

In [ ]:
from gensim.models import Word2Vec, FastText
from gensim.models import KeyedVectors

In [ ]:
model = Word2Vec([train_tok + test_tok], min_count=1)

In [ ]:
model.train([train_tok + test_tok], total_examples=model.corpus_count, epochs=1)

(10000, 990218)

In [ ]:
word_vectors = model.wv

In [ ]:
X_train = [word_vectors[w] for w in train_tok]
X_test = [word_vectors[w] for w in test_tok]

In [ ]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train, train_enc_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.5310720183331649

In [ ]:
# model = FastText([train_tok + test_tok], min_count=1)
# model.train([train_tok + test_tok], total_examples=model.corpus_count, epochs=1)
# word_vectors = model.wv
# X_train = [word_vectors[w] for w in train_tok]
# X_test = [word_vectors[w] for w in test_tok]
# lr = LogisticRegression(random_state=0)
# lr.fit(X_train, train_enc_labels)
# pred = lr.predict(X_test)
# accuracy_score(test_enc_labels, pred)

Модель тренировалась больше часа. Пришлось отказаться

### 4. взять не только эмбединги каждого слова, но и взять соседей, т.е. информацию о соседях количество соседей выбрать самим (узнать наилучшее количество соседей)

In [ ]:
vectorizers = [HashingVectorizer(ngram_range=(1, 2), analyzer='char_wb', n_features=100),
               HashingVectorizer(ngram_range=(1, 2), analyzer='word', n_features=100),
               CountVectorizer(ngram_range=(1, 2), analyzer='char_wb', max_features=100),
               CountVectorizer(ngram_range=(1, 2), analyzer='word', max_features=100),
               TfidfVectorizer(ngram_range=(1, 2), analyzer='char_wb', max_features=100),
               TfidfVectorizer(ngram_range=(1, 1), analyzer='word', max_features=100)
               ]
model = Word2Vec([train_tok + test_tok], min_count=1)
model.train([train_tok + test_tok], total_examples=model.corpus_count, epochs=1)
word_vectors = model.wv
X_train = [word_vectors[w] for w in train_tok]
X_test = [word_vectors[w] for w in test_tok]
for vectorizer in vectorizers:
  X_tn = vectorizer.fit_transform(train_tok)
  X_tt = vectorizer.transform(test_tok)
  X_train = sp.hstack((X_train, X_tn), format='csr')
  X_test = sp.hstack((X_test, X_tt), format='csr')

lr = LogisticRegression(random_state=0)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.8564856940653119

Такая модель не дала существенного улучшения качества

Попробуем другой вариант

In [ ]:
model = Word2Vec([train_tok + test_tok], min_count=1)
model.train([train_tok + test_tok], total_examples=model.corpus_count, epochs=1)
word_vectors = model.wv

In [ ]:
def w2w_ngram(n, _word_vectors, _train_tok=train_tok, _test_tok=test_tok):
  output = [sp.csr_matrix([]), sp.csr_matrix([])]
  for t, text in enumerate([_train_tok, _test_tok]):
    for i, w in enumerate(text):
      X = []
      # emb_0 = sp.csr_matrix(list(_word_vectors[text[0]]))
      # emb_l = sp.csr_matrix(list(_word_vectors[text[-1]]))
      emb_0 = list(_word_vectors[text[0]])
      emb_l = list(_word_vectors[text[-1]])
      for j in range(i - n//2, i - n//2 + n):
        if j < 0:
          emb = emb_0
        elif j >= len(text):
          emb = emb_l
        else:
          emb = list(_word_vectors[w])
        # X = sp.hstack((X, emb), format='csr')
        X.extend(emb)
      if i == 0:
        output[t] = sp.csr_matrix(X)
      else:
        output[t] = sp.vstack((output[t], X), format='csr')
  return output

In [ ]:
X_train, X_test = w2w_ngram(n=2, _word_vectors=word_vectors)

К сажалению расчет занимает больше полутора часа, выполнить не удалось.
Очень хотелось бы получить фидбэк как нужно было выполнять это задание.

In [ ]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

In [ ]:
X_train, X_test = w2w_ngram(n=3, _word_vectors=word_vectors)
lr = LogisticRegression(random_state=0)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

In [ ]:
X_train, X_test = w2w_ngram(n=4, _word_vectors=word_vectors)
lr = LogisticRegression(random_state=0)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

In [ ]:
X_train, X_test = w2w_ngram(n=5, _word_vectors=word_vectors)
lr = LogisticRegression(random_state=0)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)